# Tema 1 IA - Class Scheduler
### Alexandru LICURICEANU - 332CD

In [124]:
# Prerequisites
import utils
import itertools
#import numpy as np
from copy import copy, deepcopy
from heapq import heappop, heappush

from utils import ZILE as DAYS
from utils import INTERVALE as INTERVALS
from utils import SALI as CLASSROOMS
from utils import MATERII as SUBJECTS
from utils import PROFESORI as TEACHERS

CONSTRAINTS = 'Constrangeri'
PREFERRED = 'Preferred'
NOT_PREFERRED = 'Not_preferred'

A* approach

In [125]:
def init_empty_timetable(data):
    '''Returns an empty timetable.'''
    timetable = {day: {interval: {classroom: None for classroom in data[CLASSROOMS]} for interval in data[INTERVALS]} for day in data[DAYS]}
    return timetable

def generate_next_states(state, subject, data):
    '''Generate the neighbors of the current state.'''
    
    '''Identify all empty slots in the timetable where a subject can be assigned.
        For each empty slot, try assigning all possible combinations of teachers and subjects that satisfy the hard constraints.
        Check each assignment to ensure that it doesn't violate any of the hard constraints.
        If a valid assignment is found, create a new state representing the timetable with the assignment made.
        Repeat this process for all empty slots, generating a list of all possible neighbor states.'''
    
    next_states = []

    for day in data[DAYS]:
        for interval in data[INTERVALS]:
            for classroom in data[CLASSROOMS]:
                if state[day][interval][classroom] is None:
                    for teacher in data[TEACHERS]:
                        if subject in data[CLASSROOMS][classroom][SUBJECTS] and subject in data[TEACHERS][teacher][SUBJECTS]: 
                            new_state = deepcopy(state)
                            new_state[day][interval][classroom] = (teacher, subject)

                            if check_harder_constraints(new_state, data, day, interval, classroom, teacher, subject) == 0:
                                next_states.append(new_state)
                            

    return next_states

def check_harder_constraints(state, data, day, interval, classroom, teacher, subject):
    # Check if two classrooms overlap.
  
    # Check if a teacher is assigned to two different classrooms at the same time.

                    
    # Check if a teacher is assigned to maximum 7 classes per week.
        
    # Check if the teacher was assigned to a preferred interval and day.
    cost = 0

    if day in data[TEACHERS][teacher][NOT_PREFERRED]:
        cost += 1

    if interval in data[TEACHERS][teacher][NOT_PREFERRED]:
        cost += 1
    
    return cost
        
    

def astar(initial_state, h, is_final, data):
    state = generate_next_states(initial_state, 'PA', data)
    print(state)
    print('\n')
    

    return initial_state


In [126]:
def run_astar(input_file):
    # Read input data.
    data = utils.read_yaml_file(input_file)

    # Sort subjects by number of students.
    data[SUBJECTS] = dict(sorted(data[SUBJECTS].items(), key=lambda item: item[1]))

    preferred = {}
    not_preferred = {}

    # Process the soft constraints for each teacher.
    for teacher in data[TEACHERS]:
        preferred = [constraint for constraint in data[TEACHERS][teacher][CONSTRAINTS] if not constraint.startswith('!')]
        not_preferred = [constraint.strip('!') for constraint in data[TEACHERS][teacher][CONSTRAINTS] if constraint.startswith('!')]


        # Convert intervals from '8-12' to (8, 10) and (10, 12).
        preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in preferred if interval[0].isdigit()]
        not_preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in not_preferred if interval[0].isdigit()]

        # Split the intervals in subintervals of 2 hours.
        preferred_intervals = [interval for start, end in preferred_intervals for interval in [(start, start + 2), (start + 2, end)]]
        not_preferred_intervals = [interval for start, end in not_preferred_intervals for interval in [(start, start + 2), (start + 2, end)]]
        
        # Remove intervals that have same start and end time.
        preferred_intervals = [str(interval) for interval in preferred_intervals if interval[0] != interval[1]]
        not_preferred_intervals = [str(interval) for interval in not_preferred_intervals if interval[0] != interval[1]]


        # Create the preferred and not preferred days.
        preferred_days = [day for day in preferred if not day[0].isdigit()]
        not_preferred_days = [day.strip('!') for day in not_preferred if not day[0].isdigit()]
        
        data[TEACHERS][teacher][PREFERRED] = preferred_days + preferred_intervals
        data[TEACHERS][teacher][NOT_PREFERRED] = not_preferred_days + not_preferred_intervals

    
    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

    # Run the A* algorithm.
    result = astar(initial_state, None, None, data)
    return result

In [127]:
input_files = ['orar_mic_exact.yaml']

def transform_intervals(timetable):
    '''Function that transforms the intervals of the timetable from strings to tuples,
    in order to work with the utils.pretty_print function.'''

    new_timetable = {}

    for day in timetable:
        new_timetable[day] = {}

        for interval in timetable[day]:
            temp = interval.strip('()').split(',')
            new_interval = (int(temp[0]), int(temp[1]))
            new_timetable[day][new_interval] = timetable[day][interval]

    return new_timetable

def test_astar():
    for input_file in input_files:
        result = run_astar('inputs/' + input_file)

        with open('outputs/' + input_file.replace('.yaml', '.txt'), 'w') as f:
           f.write(utils.pretty_print_timetable(transform_intervals(result), 'inputs/' + input_file))

test_astar()

[{'Luni': {'(8, 10)': {'ED010': ('Andrei Moldovan', 'PA'), 'ED020': None}, '(10, 12)': {'ED010': None, 'ED020': None}, '(12, 14)': {'ED010': None, 'ED020': None}, '(14, 16)': {'ED010': None, 'ED020': None}, '(16, 18)': {'ED010': None, 'ED020': None}, '(18, 20)': {'ED010': None, 'ED020': None}}, 'Marti': {'(8, 10)': {'ED010': None, 'ED020': None}, '(10, 12)': {'ED010': None, 'ED020': None}, '(12, 14)': {'ED010': None, 'ED020': None}, '(14, 16)': {'ED010': None, 'ED020': None}, '(16, 18)': {'ED010': None, 'ED020': None}, '(18, 20)': {'ED010': None, 'ED020': None}}, 'Miercuri': {'(8, 10)': {'ED010': None, 'ED020': None}, '(10, 12)': {'ED010': None, 'ED020': None}, '(12, 14)': {'ED010': None, 'ED020': None}, '(14, 16)': {'ED010': None, 'ED020': None}, '(16, 18)': {'ED010': None, 'ED020': None}, '(18, 20)': {'ED010': None, 'ED020': None}}, 'Joi': {'(8, 10)': {'ED010': None, 'ED020': None}, '(10, 12)': {'ED010': None, 'ED020': None}, '(12, 14)': {'ED010': None, 'ED020': None}, '(14, 16)': {'